<a href="https://colab.research.google.com/github/Vish4github/Projects/blob/master/News%20Category%20Classification%20using%20Transfer%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install fastai==0.7.0

In [0]:
!pip3 install torchvision
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [0]:
from fastai.imports import *

In [0]:
from torch import * 
!pip install Pillow==4.1.1
!pip install image
%matplotlib inline
from fastai.imports import *

In [0]:
!pip install "torchtext==0.2.3"

In [0]:
!pip3 install spacy 
!python -m spacy download en

In [0]:
!nvcc --version

In [0]:
import torch
torch.cuda.is_available()
#torch.backends.cudnn.enabled
#!nvidia-smi
#!python --version

In [0]:
#!pip install fastai
!pip install fastai --upgrade


In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai import *
from fastai.text import *
from fastai.text import Tokenizer, Vocab
from keras.preprocessing.text import Tokenizer, text_to_word_sequence

In [0]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/Data/News_Category_Dataset_v2.json"
df =  pd.read_json(path,lines=True)
df.head()

In [0]:
cates = df.groupby('category')
#print("total categories:", cates.ngroups)
#print(cates.size())
df.category = df.category.map(lambda x: "WORLDPOST" if x == "THE WORLDPOST" else x)

In [0]:
# using headlines and short_description as input X

df['text'] = df.headline + " " + df.short_description

# tokenizing

tokenizer = Tokenizer()
#tokenizer.process_text(df.text)
tokenizer.fit_on_texts(df.text)
X = tokenizer.texts_to_sequences(df.text)
df['words'] = X

# delete some empty and short data

df['word_length'] = df.words.apply(lambda i: len(i))
df = df[df.word_length >= 5]



In [0]:
df.head()
df = df.drop(df.columns[[1,2,3,4,5,7,8]], axis=1)
df.shape
df.to_csv('/content/drive/My Drive/Data/cleaned.csv',encoding='utf-8')

In [0]:
df2 = pd.read_csv('/content/drive/My Drive/Data/cleaned.csv')
df3 = df2
df3.head()

X=df3.sample(frac=0.85,random_state=200)
test=df3.drop(X.index)
train=X.sample(frac=0.85,random_state=200)
val=X.drop(train.index)

print(train.shape)
print(val.shape)
print(test.shape)
train.to_csv('/content/drive/My Drive/Data/train.csv',encoding='utf-8',index=False)
val.to_csv('/content/drive/My Drive/Data/val.csv',encoding='utf-8',index=False)
test.to_csv('/content/drive/My Drive/Data/test.csv',encoding='utf-8',index=False)

df3.head()

In [0]:
train_df=pd.read_csv('/content/drive/My Drive/Data/train.csv')
train_df=train_df[['category','text']]

In [0]:
valid_df=pd.read_csv('/content/drive/My Drive/Data/val.csv')
valid_df=valid_df[['category','text']]

In [0]:
#from sklearn.model_selection import train_test_split
#train_df,valid_df=train_test_split(df_text, shuffle=True, test_size=0.25)
len(train_df),len(valid_df)

In [0]:
train_df.head(),valid_df.head()

In [0]:
bs,bptt=64,60

A text is,unlike images which can be fed directly into a model, is composed of words, and we can't apply mathematical functions to them directly. We first have to convert them to numbers. This is done in two differents steps: tokenization and numericalization. A TextDataBunch does all of that behind the scenes.

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(path,train_df=train_df,valid_df=valid_df,text_cols='text', label_cols='category',bs=bs)

# Classification model data
data_clas = TextClasDataBunch.from_df(path, train_df=train_df,valid_df=valid_df, vocab=data_lm.train_ds.vocab, text_cols = 'text', label_cols = 'category',bs=bs)


Once we have extracted tokens from our texts, we convert to integers by creating a list of all the words used ( Vocab or dictionary). We only keep the ones that appear atleast twice with a maximum vocabulary size of 60,000 (by default) and the ones that don't make the cut are tagged as UNK

Once we have extracted tokens from our texts, we convert to integers by creating a list of all the words used ( Vocab or dictionary). We only keep the ones that appear atleast twice with a maximum vocabulary size of 60,000 (by default) and the ones that don't make the cut are tagged as UNK

.itos and .stoi will give us information on how our tokens and numericalized and stored 

In [0]:
data_lm.vocab.itos[:10]  # first 10 tokens in our vocab

In [0]:
data_lm.vocab.itos[448]

In [0]:
data_lm.train_ds[0][0].data[:10]  # first sentence in our training dataset

In [0]:
len(data_lm.vocab.itos),len(data_lm.train_ds)

In [0]:
data_clas.show_batch()  # shows a portion of the data loaded for the huffpost language model

The intention is not to train a model that classifies the news categories from scratch. Like in computer vision, we'll use a model pretrained on a bigger dataset (a cleaned subset of wikipedia called wikitext-103). That model has been trained to guess what the next word, its input being all the previous words. It has a recurrent structure and a hidden state that is updated each time it sees a new word. This hidden state thus contains information about the sentence up to that point.

We are going to use that 'knowledge' of the English language to build our classifier, but first, like for computer vision, we need to fine-tune the pretrained model to our particular dataset. Because the English of the news headlines and descriptions on huffpost isn't the same as the English of wikipedia, we'll need to adjust a little bit the parameters of our model.Also there might be some words extremely common in that dataset that were barely present in wikipedia, and therefore might no be part of the vocabulary the model was trained on.

In [0]:
#Saving the datasets
data_lm.save('/content/drive/My Drive/Data/data_lm_export.pkl')
data_clas.save('/content/drive/My Drive/Data/data_clas_export.pkl')

In [0]:
path = '/content/drive/My Drive/Data/'

In [0]:
data_lm = load_data(path, 'data_lm_export.pkl',bs=bs,bptt=bptt)   #bptt = back propagation through time
data_clas = load_data(path, 'data_clas_export.pkl', bs=bs)
data_bwd = load_data(path, 'data_lm_export.pkl', bs=bs,bptt=bptt, backwards=True)

In [0]:
data_bwd.show_batch()

Fine tuning the language model

The idea behind the ULMFit paper is to use transfer learning for this classification task. Our language model isn't randomly initialized but with the weights of a model pretrained on a larger corpus, Wikitext 103. The vocabulary of the two datasets are slightly different, so when loading the weights, we take care to put the embedding weights at the right place, and we rando;ly initiliaze the embeddings for words in the huff post vocab that weren't in the wikitext-103 vocabulary of our pretrained model.

This is all done by the first line of code that will download the pretrained model at the first use. The second line is to use Mixed Precision Training, which enables us to use a higher batch size by training part of our model in FP16 precision, and also speeds up training by a factor 2 to 3 on modern GPUs.

In [0]:
learn_model = language_model_learner(data_lm, AWD_LSTM,drop_mult=0.3)

In [0]:
#learn_model = learn_model.to_fp16(clip=0.1)

In [0]:
learn_model.lr_find() # to find appropriate LR

In [0]:
learn_model.recorder.plot(skip_end=15) # to plot the LR

In [0]:
lr = 1e-2
lr *= bs/48

In [14]:
learn_model.fit_one_cycle(1, 1e-2, moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,time
0,3.887925,3.716582,0.394290,18:22


Now we unfreeze the layers, not just one as in the previous step

In [0]:
learn_model.unfreeze()

In [0]:
learn_model.fit_one_cycle(6, lr, moms=(0.8,0.7), wd=0.1)  # we can adjust the learning rate and run the model again if required

In [0]:
#learn_model.fit_one_cycle(1, lr*10, moms=(0.8,0.7))

Now we will save just the encoder which will be later used for classification

In [0]:
learn_model.save_encoder('fwd_enc')
#learn_lm.save('fit_1')  # saving the fine tuned language model
#learn_lm.load('fit_1')

Backwards model

You can't directly train a bidirectional RNN for language modeling, but you can always enseble a forward and backward model. fastai provides a pretrained forward and backawrd model, so we can repeat the previous step to fine-tune the pretrained backward model. The command language_model_learner checks the data object you pass to automatically decide if it should use the pretrained forward or backward model.

In [0]:
learn_model = language_model_learner(data_bwd, AWD_LSTM,drop_mult=0.3)

In [0]:
learn_model.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)

In [0]:
learn_model.unfreeze()

In [0]:
learn_model.fit_one_cycle(6, lr, moms=(0.8,0.7),wd=0.1)

In [0]:
learn_model.save_encoder('bwd_enc')

In [0]:
vocab = data_lm.vocab

In [0]:
vocab.stoi["stingray"]


In [0]:
awd = learn_model.model[0]

In [0]:
enc = learn_model.model[0].encoder
enc.weight.size()

In [0]:
vocab.stoi["trump"]

In [0]:
TEXT = "Trump is"
N_WORDS = 30
N_SENTENCES = 1

In [0]:
print("\n".join(learn_model.predict(TEXT, N_WORDS, temperature=0.75) for _ in range(N_SENTENCES)))  #wont make sense since this is the output from the backward model

In [0]:
learn_model.predict("This is a review about", n_words=20)

Classifier

In [0]:
bs=32

In [0]:
data_clas.show_batch()

In [0]:
data_clas_bwd = load_data(path, 'data_clas_export.pkl', bs=bs, backwards=True)   #loading data for the backward classifier

In [0]:
data_clas_bwd.show_batch()

Fine tuning the forward classifier

The classifier needs a little less dropout, so we pass drop_mult=0.5 to multiply all the dropouts by this amount (it's easier than adjusting all the five different values manually). We don't load the pretrained model, but instead our fine-tuned encoder from the previous section.

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn.load_encoder('fwd_enc')

In [0]:
data_clas.show_batch()

In [0]:
lr = 1e-2
learn.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)

In [0]:
learn.freeze_to(-2)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

In [0]:
learn.save('last2')

In [0]:
learn.load('last2')

In [0]:
learn.freeze_to(-3)
lr /= 2
learn.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

In [0]:
learn.save('last3')

In [0]:
learn.load('last3')

In [0]:
learn.unfreeze()
lr /= 5
learn.fit_one_cycle(10, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

In [0]:
learn.save('fwd_clas')

In [62]:
preds,targs = learn.get_preds(ordered=True)

In [63]:
accuracy(preds,targs)

tensor(0.6967)

In [0]:
learn.load('fwd_clas')

Same thing backwards

Then we do the same thing for the backward model, the only thigns to adjust are the names of the data object and the fine-tuned encoder we load.

In [0]:
learn_bwd = text_classifier_learner(data_clas_bwd, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn_bwd.load_encoder('bwd_enc')

In [0]:
lr = 1e-2
learn_bwd.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)

In [0]:
learn_bwd.freeze_to(-2)
lr /= 2
learn_bwd.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

In [0]:
learn_bwd.save('twolayers_bwd')

In [0]:
learn_bwd.load('twolayers_bwd')

In [0]:
learn_bwd.freeze_to(-3)
lr /= 2
learn_bwd.fit_one_cycle(1, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

In [0]:
learn_bwd.save('threelayer_bwd')

In [0]:
learn_bwd.load('threelayer_bwd')

In [0]:
learn_bwd.unfreeze()
lr /= 5
learn_bwd.fit_one_cycle(10, slice(lr/(2.6**4),lr), moms=(0.8,0.7), wd=0.1)

In [74]:
preds_bwd,targs_bwd = learn_bwd.get_preds(ordered=True)
accuracy(preds_bwd,targs_bwd)

tensor(0.6994)

In [0]:
learn_bwd.save('bwd_clas')

Ensembling the two models


For our final results, we'll take the average of the predictions of the forward and the backward models. SInce the samples are sorted by text lengths for batching, we pass the argument ordered=True to get the predictions in the order of the texts.

In [0]:
pred_fwd,lbl_fwd = learn.get_preds(ordered=True)

In [0]:
pred_bwd,lbl_bwd = learn_bwd.get_preds(ordered=True)

In [0]:
final_pred = (pred_fwd+pred_bwd)/2

In [0]:
accuracy(final_pred, lbl_fwd)
#tensor(0.7141)